# RAG CHATBOT ENFOCADO EN FAQ PARA ASISTENTE TECNICO


## Cargar PDFS

In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage

In [2]:
f = open("C:\\Users\\diego\\OneDrive\\Documents\\ApiKey.txt")
#Este txt tiene que tener el string en un archivo de texto creado previamente en la plataforma de OPENAI
api_key = f.read()
chat = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=api_key)


In [3]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

In [4]:
def load_pdfs(pdf_folder_path):
    pdf_folder = Path(r"C:\Users\diego\OneDrive\Documents\Manuales Mcdonalds\MANUALES")
    pdf_files = list(pdf_folder.glob("*.pdf"))
          
    all_documents = []

    for pdf_file in pdf_files:
        loader = PyPDFLoader(str(pdf_file))
        docs = loader.load()

        # agregar metadata
        for d in docs:
            d.metadata["source_file"] = pdf_file.name

        all_documents.extend(docs)
    
    return all_documents

In [5]:
all_documents = load_pdfs(r"C:\Users\diego\OneDrive\Documents\Manuales Mcdonalds\MANUALES")


Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 75 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 91 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 215 0 (offset 0)
Ignoring wrong pointing object 267 0 (offset 0)


## Chunking

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
def split_chunks(all_documents, chunk_size=900, overlap=120):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        length_function=len
    )
    return splitter.split_documents(all_documents)

In [8]:
chunks = split_chunks(all_documents)

In [9]:
#chunks[4].page_content

In [77]:
#len(chunks[2].page_content)

712

In [76]:
#len(chunks)

198

In [12]:
#print(chunks[112].metadata)


In [13]:
#for i in range(1):
    #if len(chunks[i].page_content) > 50:
        #print("======CHUNK PARA RAG===========: ", i,"\n") 
        #print(chunks[i].page_content)
    #else:
        #print("------------------------------------CHUNK PARA VISION:", i , "-------------------------------------")
        #print(chunks[i].page_content)
        #print(chunks[i].metadata)

In [14]:
#print(chunks[197].metadata)


In [15]:
chunks_rag=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 
            25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
            47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 69, 70, 71,
            72, 76, 77, 81, 82, 83, 84, 86, 87, 91, 93, 95, 100, 101, 102, 105, 108, 109, 110, 111,
            153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 167, 168, 169, 171, 172, 173, 174, 175,
            176, 177, 178, 179, 180, 181, 183, 184, 185, 187, 190, 191, 192, 193, 194, 195, 196, 197]
chunks_vision=[59,67,68,74,75,78,79,80,85,88,89,90,92,94,96,97,98,99,103,104,106,107,112,113,114,
              115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 
              132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
              150, 151, 152,163,186,188,189]
chunks_eliminar=[73,164,165,166,170,182]


def filter_chunks(chunks, chunks_rag, chunks_vision, chunks_eliminar):
    rag = []
    vision = []
    removed = []

    for i, ch in enumerate(chunks):
        if i in chunks_rag:
            rag.append(ch)
        elif i in chunks_vision:
            vision.append(ch)
        elif i in chunks_eliminar:
            removed.append(ch)
    
    return chunks_rag, chunks_vision, chunks_eliminar

In [16]:
print("rag", chunks_rag)

rag [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 66, 69, 70, 71, 72, 76, 77, 81, 82, 83, 84, 86, 87, 91, 93, 95, 100, 101, 102, 105, 108, 109, 110, 111, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 183, 184, 185, 187, 190, 191, 192, 193, 194, 195, 196, 197]


In [17]:
count = 0
for i in chunks_rag:
    count += 1
    
print("El total de chunks es", count)
    

El total de chunks es 124


In [18]:
#type(chunks_rag)

In [19]:
#type(chunks_rag[0])

In [20]:
def build_rag_and_vision_lists(chunks, chunks_rag, chunks_vision):
    rag_chunks = [chunks[i] for i in chunks_rag]
    vision_chunks = [chunks[i] for i in chunks_vision]
    return rag_chunks, vision_chunks

In [21]:
rag_chunks, vision_chunks = build_rag_and_vision_lists(chunks, chunks_rag, chunks_vision)

In [22]:
#type(rag_chunks)

In [23]:
#type(rag_chunks[0])

## Embeddings

In [24]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import CSVLoader

In [25]:
def build_embeddings(rag_chunks):
    embedder = OpenAIEmbeddings(
        model="text-embedding-3-small",
        openai_api_key=api_key
    )
    embedded_docs = embedder.embed_documents(
        [doc.page_content for doc in rag_chunks]
    )
    return embedded_docs


In [26]:
embedded_docs = build_embeddings(rag_chunks)


### Pruebas para verificar embeddings correctos: 

### Tamaño del vector:

In [27]:
for emb in embedded_docs:
    print(len(emb))

1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536
1536


###  Prueba de Longitud:

In [28]:
dims= set(len(vec) for vec in embedded_docs)

print(dims)

{1536}


### Prueba semántica:

In [29]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

print(cosine_similarity(embedded_docs[78], embedded_docs[6]))


0.5337184919928519


# Creamos Vector Store para guardar en Disco 

In [30]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

def build_vector_store(rag_chunks, api_key, store_name="faiss_store_mcdonalds"):
    #Embedding model
    embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key)


    #Crear vector store desde documentosr
    store = FAISS.from_documents(rag_chunks, embedding_model)

    #Guardar local
    store.save_local(store_name)

    #Recargar (buena práctica para producción)
    store = FAISS.load_local(
        store_name,
        embedding_model,
        allow_dangerous_deserialization=True)

    #Crear retriever
    retriever = store.as_retriever(search_type="similarity", k=3)

    return store, retriever


In [31]:
store, retriever = build_vector_store(rag_chunks, api_key)


In [32]:
def test_retrieval(query):
    docs = retriever.invoke(query)

    print("\n QUERY:", query)
    print("=" * 80)

    for idx, d in enumerate(docs):
        print(f"\n  CHUNK {idx} (len={len(d.page_content)})")
        print(d.page_content)
        print("-" * 80)
        print("\n -----METADATOS----")



In [33]:
test_retrieval("¿Cómo configurar una impresora EVERSERV?")



 QUERY: ¿Cómo configurar una impresora EVERSERV?

  CHUNK 0 (len=390)
ELABORÓ:  Dirección  de  Operaciones  
Scanner
 Scanner
 
  
Comentarios:   conﬁrmamos  la  conﬁguración  de  nuestro  puerto  
Comentarios:    conﬁrmamos  la  conﬁguración  de  nuestro  puerto  
Scanner
 Scanner
 
  
Comentarios:   ingresamos  a  panel  de  control/impresoras  y  agregamos  nuestra  impresora  
Comentarios:  seleccionamos  el  driver  instalado  de  nuestra  impresaora.
--------------------------------------------------------------------------------

 -----METADATOS----

  CHUNK 1 (len=776)
____________________________ Documentación y Manuales  _______________________________  2  
  
Procedimiento para configurar impresora Ithaca 9000-sl 
 
Las impresoras traen de fabrica una configuración la cual podemos modificar con el siguiente procedimiento. 
 
1. Oprimir el botón FEED (Avance de Papel) con la impresora apagada, sin soltarlo la encendemos, se iluminará de 
color verde el tercer led, esperamos 

In [34]:
#query = "¿Cómo desconectar el botón de encendido del Everserv 600?"

#results = retriever.invoke(query)

#print(f" Se recuperaron {len(results)} chunks \n")

#for i, doc in enumerate(results):
    #print(f"====== CHUNK{i+1}=======")
    #print("Longitud", len(doc.page_content))
    #print(doc.page_content)
    #print("\n -----METADATOS----")
    #print(doc.metadata)
    #print("\n\n")

In [35]:
#def test_retrieval(query):
    #docs = retriever.invoke(query)

    #print("\n QUERY:", query)
    #print("=" * 80)

    #for idx, d in enumerate(docs):
        #print(f"\n  CHUNK {idx} (len={len(d.page_content)})")
        #print(d.page_content)
        #print("-" * 80)
        #print("\n -----METADATOS----")
        #print(doc.metadata)
        #print("\n\n")

#Ejemplo
#test_retrieval("¿Cómo configurar una impresora NGK?")

In [36]:
from langchain_core.prompts import ( HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, PromptTemplate, )

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=300,
    openai_api_key=api_key
)


In [37]:
system_template = """Eres un asistente técnico especializado en soporte de hardware y software para personal de IT.
Respondes con un tono cordial, casual, educado y directo.

Reglas estrictas:
- Usa ÚNICAMENTE la información proporcionada en el contexto que se te entregue.
- No inventes información ni hagas suposiciones.
- Comprende preguntas con errores ortográficos, abreviaciones y jerga popular mexicana, sin corregirlas ni comentarlas.
- Cuando sea posible, responde usando pasos numerados o viñetas.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)


In [38]:
human_template = """
Contexto:
{context}

Pregunta del usuario:
{question}

Respuesta:
"""
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [39]:
human_message_prompt.input_variables

['context', 'question']

In [40]:
#Plantilla de chat con la concatenacion tanto de mensaje de sistema como del humano
chat_prompt=ChatPromptTemplate.from_messages([system_message_prompt,human_message_prompt])
chat_prompt.input_variables

['context', 'question']

In [41]:
def ask_assistant(question):

    docs = retriever.invoke(question)

    context = "\n\n".join(doc.page_content for doc in docs)

    messages = chat_prompt.format_messages(
        context=context,
        question=question
    )

    response = llm.invoke(messages)

    print("\nRESPUESTA:\n")
    print(response.content)


In [78]:
ask_assistant("Hola amigo, dime como arreglar una impresora ithaca 9000?")



RESPUESTA:

¡Hola! Para arreglar una Everserv 600, aquí tienes algunos pasos que puedes seguir:

1. **Revisar conexiones**:
   - Asegúrate de que todos los cables estén bien conectados, especialmente el cable del botón de encendido y el cable de la membrana touch.

2. **Desmontar la carcasa**:
   - Retira los tornillos para desmontar la carcasa de la base. Esto te permitirá acceder a los componentes internos.

3. **Inspeccionar componentes**:
   - Verifica si hay derrames de líquidos que puedan haber afectado las conexiones.
   - Revisa si algún engrane se ha soltado y vuelve a montarlo. Si hay daño en los engranes, considera cambiar la impresora por completo.

4. **Mantenimiento del POS**:
   - Realiza limpieza de componentes y de las partes plásticas.
   - Asegúrate de que el equipo esté bien ensamblado y reconectado.

5. **Configuración del scanner**:
   - Ingresa como administrador y abre el emulador Epson desde tu USB.
   - Instala el driver y valida en el puerto asignado, config

In [43]:
ask_assistant("ya hice los pasos que me dijiste, pero no se porque sigue parpadenado rojo, que hago?")


RESPUESTA:

Claro, aquí tienes algunos pasos que puedes seguir para solucionar el problema del parpadeo rojo:

1. **Verifica la conexión**: Asegúrate de que todos los cables estén bien conectados y que no haya daños visibles.

2. **Revisa la configuración de red**:
   - Asegúrate de haber ingresado correctamente la IP, máscara, puerta y DNS.
   - Si no tienes estos datos, verifica en las pantallas de alado.

3. **Confirma la URL**:
   - Asegúrate de que la URL ingresada sea correcta: `rmmcd.dexmanager.com`.
   - Verifica que el puerto esté configurado en 443.

4. **Revisa la configuración del puerto**:
   - Asegúrate de que el puerto COM esté correctamente asignado (debería ser COM12 si seguiste los pasos).

5. **Reinicia el dispositivo**: A veces, un reinicio puede ayudar a aplicar los cambios realizados.

Si después de seguir estos pasos el problema persiste, por favor proporciona más detalles sobre el dispositivo o cualquier mensaje de error que aparezca. ¡Estoy aquí para ayudarte!

# Reconocimiento de imagenes 

In [44]:
from pathlib import Path
import fitz  # PyMuPDF

def extract_images_from_pdfs(pdf_folder_path, output_folder="extracted_images"):
    pdf_folder = Path(pdf_folder_path)
    out_dir = Path(output_folder)
    out_dir.mkdir(parents=True, exist_ok=True)

    images_metadata = []  # aquí guardamos info de cada imagen

    pdf_files = list(pdf_folder.glob("*.pdf"))
    print("PDFs encontrados:", len(pdf_files))

    for pdf_path in pdf_files:
        doc = fitz.open(pdf_path)

        for page_index in range(len(doc)):
            page = doc[page_index]
            image_list = page.get_images(full=True)

            if not image_list:
                continue

            for img_num, img in enumerate(image_list):
                xref = img[0]
                base_image = doc.extract_image(xref)
                image_bytes = base_image["image"]
                ext = base_image.get("ext", "png")

                # nombre único: archivo__página__img
                img_name = f"{pdf_path.stem}__p{page_index+1}__img{img_num+1}.{ext}"
                img_path = out_dir / img_name

                with open(img_path, "wb") as f:
                    f.write(image_bytes)

                images_metadata.append({
                    "image_path": str(img_path),
                    "source_file": pdf_path.name,
                    "page": page_index + 1,
                    "img_index": img_num + 1
                })

        doc.close()

    print("Imágenes extraídas:", len(images_metadata))
    return images_metadata


In [45]:
images_metadata = extract_images_from_pdfs(
    r"C:\Users\diego\OneDrive\Documents\Manuales Mcdonalds\MANUALES",
    output_folder="manual_images"
)


PDFs encontrados: 14
Imágenes extraídas: 918


In [46]:
images_metadata[:5]


[{'image_path': 'manual_images\\Capacitacion_1__p1__img1.png',
  'source_file': 'Capacitacion_1.pdf',
  'page': 1,
  'img_index': 1},
 {'image_path': 'manual_images\\Capacitacion_1__p1__img2.png',
  'source_file': 'Capacitacion_1.pdf',
  'page': 1,
  'img_index': 2},
 {'image_path': 'manual_images\\Capacitacion_1__p1__img3.png',
  'source_file': 'Capacitacion_1.pdf',
  'page': 1,
  'img_index': 3},
 {'image_path': 'manual_images\\Capacitacion_1__p2__img1.png',
  'source_file': 'Capacitacion_1.pdf',
  'page': 2,
  'img_index': 1},
 {'image_path': 'manual_images\\Capacitacion_1__p2__img2.png',
  'source_file': 'Capacitacion_1.pdf',
  'page': 2,
  'img_index': 2}]

### Filtrado de imagenes 

In [47]:
from PIL import Image

def filter_useful_images(
    images_metadata,
    vision_pages_by_pdf,
    min_width=300,
    min_height=300
):
    """
    vision_pages_by_pdf = {
        "Everserv_600_desarmado.pdf": [3, 4, 5],
        "Everserv_8000_guiarapida.pdf": [1, 2]
    }
    """

    filtered_images = []

    for img in images_metadata:
        pdf = img["source_file"]
        page = img["page"]
        img_path = img["image_path"]

        if pdf not in vision_pages_by_pdf:
            continue

        if page not in vision_pages_by_pdf[pdf]:
            continue

        try:
            with Image.open(img_path) as im:
                width, height = im.size
        except Exception:
            continue

        if width < min_width or height < min_height:
            continue

        img["width"] = width
        img["height"] = height
        filtered_images.append(img)

    print("Imágenes útiles:", len(filtered_images))
    return filtered_images


In [48]:
vision_pages_by_pdf = {
    "Capacitacion_1.pdf": [2, 3, 4],
    "Capacitacion_2.pdf": [2, 3, 4,5,6,7,8],
    "Capacitacion_3.pdf": [2, 3, 4,5],
    "Capacitacion_4.pdf": [2, 3, 4,5,6,7,8,9,10,11,12,13],
    "Capacitacion_5.pdf": [2, 3, 4,5,6,7,8,9,10,11,12,13,14,15,16,17],
    "Everserv_600_desarmado.pdf": [1,2, 3, 4,5,6],
    "Everserv_600V2_conexion.pdf": [1,2,3],
    "Everserv_WAVE_desarmado.pdf": [2,3,4,5,6],
    "Hardware_FSS_instalacion.pdf": [2,6,7,11,12,16,17,18,22,23,24,27,28,29,30,31,32,33,34,35,36,40,41,42,43],
    "Ithaca_9000_desarmado.pdf": [2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,
                                  35,36,37,38,39,40,41,42,43,44,45,46,47],
    "Ithaca_9000_manual.pdf": [2,3,4,5,6,7],
    "Manual_mantenimientos_preventivos_correctivo.pdf": [5,6,7,8,9,10,11,12,13,14,15,16,17],
    "NGK'S.pdf": [2,3,4,5,6,7,8,9],
    "Par_Ever_8000_conexion.pdf": [2,3,4,5]  
}


In [49]:
useful_images = filter_useful_images(
    images_metadata=images_metadata,
    vision_pages_by_pdf=vision_pages_by_pdf,
    min_width=300,
    min_height=300
)

Imágenes útiles: 294


In [58]:
import os
from openai import OpenAI

with open(r"C:\Users\diego\OneDrive\Documents\ApiKey.txt", "r") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

client = OpenAI()


In [59]:
import base64, json, re
from openai import OpenAI

client = OpenAI()

def bytes_to_data_url(image_bytes: bytes, mime: str = "image/jpeg") -> str:
    b64 = base64.b64encode(image_bytes).decode("utf-8")
    return f"data:{mime};base64,{b64}"


VISION_PROMPT = """
Eres un técnico TI. Observa la imagen y devuelve SOLO JSON válido.

Formato:
{
  "device_type": "",
  "brand_model": "",
  "screen_text": "",
  "error_code_or_message": "",
  "led_pattern": "",
  "what_you_see": [],
  "confidence": "alta|media|baja"
}

Si algo no se ve claramente, deja el campo vacío.
"""

def analyze_tech_photo_bytes(image_bytes: bytes, user_text: str, mime="image/jpeg"):
    data_url = bytes_to_data_url(image_bytes, mime=mime)

    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": VISION_PROMPT},
            {"role": "user", "content": [
                {"type": "text", "text": user_text},
                {"type": "image_url", "image_url": {"url": data_url}}
            ]}
        ],
    )

    text = resp.choices[0].message.content.strip()
    json_str = re.search(r"\{.*\}", text, re.DOTALL).group()
    return json.loads(json_str)



In [60]:
def build_query(user_text, vision):
    return f"""
impresora error soporte tecnico
no imprime
led rojo parpadeo

LED: {vision.get("led_pattern","")}
PANTALLA: {vision.get("screen_text","")}
CODIGO: {vision.get("error_code_or_message","")}
MODELO: {vision.get("brand_model","")}

EVITAR: scanner escaner puerto configuracion
"""


In [61]:
def has_visible_error(vision):
    return any([
        vision.get("led_pattern"),
        vision.get("screen_text"),
        vision.get("error_code_or_message")
    ])


In [62]:
retriever = store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8, "fetch_k": 30}
)


In [63]:
def has_real_evidence(docs):
    banned = ["scanner", "escaner", "puerto"]
    for d in docs:
        txt = (d.page_content or "").lower()
        if not any(b in txt for b in banned):
            return True
    return False


In [64]:
def answer_ticket_bytes(user_text, image_bytes, retriever, llm, mime="image/jpeg"):
    vision = analyze_tech_photo_bytes(image_bytes, user_text, mime=mime)

    if vision.get("confidence") == "alta" and not has_visible_error(vision):
        return {
            "status": "no_error_detected",
            "message": (
                "En la imagen no se observa ningún error activo.\n\n"
                "La impresora aparece con la tapa abierta y el mecanismo visible, "
                "lo cual es normal durante la carga o revisión de papel.\n\n"
                "Si el problema es que no imprime, confirma:\n"
                "1) Que la tapa esté completamente cerrada\n"
                "2) Que haya papel cargado correctamente\n"
                "3) Si al encenderla aparece algún LED o mensaje de error"
            ),
            "vision": vision
        }

    if vision.get("confidence") == "baja":
        return {
            "status": "need_more_info",
            "message": (
                "No puedo identificar un problema técnico claro solo con la imagen.\n\n"
                "Para continuar necesito:\n"
                "1) Modelo exacto del equipo\n"
                "2) ¿Hay algún LED encendido o parpadeando?\n"
                "3) ¿Aparece algún mensaje o código?\n"
                "4) ¿Está conectado por USB o red?"
            ),
            "vision": vision
        }

    query = build_query(user_text, vision)
    docs = retriever.invoke(query)

    if not docs:
        return {
            "status": "need_more_info",
            "message": (
                "No encontré información suficiente en los manuales con los datos actuales.\n"
                "Proporciona una imagen donde se vea un LED, mensaje o código de error."
            ),
            "vision": vision
        }

    context = "\n\n".join(
        [f"[{i}] {d.page_content}" for i, d in enumerate(docs)]
    )

    prompt = f"""
Eres un asistente técnico TI.

REGLAS:
- Usa SOLO el contexto.
- No inventes pasos.
- Si no hay evidencia, dilo.

CONTEXTO:
{context}

PROBLEMA:
{user_text}

SEÑALES VISIBLES:
{vision}
"""

    answer = llm.invoke(prompt)

    return {
        "status": "solution",
        "answer": answer,
        "vision": vision
    }


In [65]:
result = answer_ticket(
    user_text="La impresora no imprime",
    image_path=r"C:\Users\diego\Downloads\impresora.png",
    retriever=re|triever,
    llm=llm
)

print(result)


{'status': 'no_error_detected', 'message': 'En la imagen no se observa ningún error activo.\n\nLa impresora aparece con la tapa abierta y el mecanismo visible, lo cual es normal durante la carga o revisión de papel.\n\nSi el problema es que no imprime, confirma:\n1) Que la tapa esté completamente cerrada\n2) Que haya papel cargado correctamente\n3) Si al encenderla aparece algún LED o mensaje de error', 'vision': {'device_type': 'impresora', 'brand_model': 'Ithaca 900', 'screen_text': '', 'error_code_or_message': '', 'led_pattern': '', 'what_you_see': ['compartimento abierto', 'rodillo visible', 'sin papel'], 'confidence': 'alta'}}


# PRODUCCIÓN RAG

### *el bot se pasa a vs para mandarlo a producción*